In [1]:
import pandas as pd
import os

In [2]:
raw_data_folder = 'Project_Rawdata/'

In [3]:
csv_files = [f for f in os.listdir(raw_data_folder) if f.endswith('.csv')]

In [4]:
print(f"Found {len(csv_files)} CSV files in '{raw_data_folder}':")
for f in csv_files:
    print(f"- {f}")

Found 6 CSV files in 'Project_Rawdata/':
- 202501-divvy-tripdata.csv
- 202502-divvy-tripdata.csv
- 202503-divvy-tripdata.csv
- 202504-divvy-tripdata.csv
- 202505-divvy-tripdata.csv
- 202506-divvy-tripdata.csv


In [5]:
all_dfs = []

In [6]:
for file_name in csv_files:
    file_path = os.path.join(raw_data_folder, file_name)
    print(f"Reading {file_path}...")
    df = pd.read_csv(file_path, low_memory=False)
    all_dfs.append(df)

Reading Project_Rawdata/202501-divvy-tripdata.csv...
Reading Project_Rawdata/202502-divvy-tripdata.csv...
Reading Project_Rawdata/202503-divvy-tripdata.csv...
Reading Project_Rawdata/202504-divvy-tripdata.csv...
Reading Project_Rawdata/202505-divvy-tripdata.csv...
Reading Project_Rawdata/202506-divvy-tripdata.csv...


In [7]:
print("Concatenating all DataFrames...")
combined_df = pd.concat(all_dfs, ignore_index=True)

Concatenating all DataFrames...


In [8]:
print(f"Combined DataFrame shape: {combined_df.shape}")

Combined DataFrame shape: (2141425, 13)


In [9]:
print(combined_df.head())

            ride_id  rideable_type               started_at  \
0  7569BC890583FCD7   classic_bike  2025-01-21 17:23:54.538   
1  013609308856B7FC  electric_bike  2025-01-11 15:44:06.795   
2  EACACD3CE0607C0D   classic_bike  2025-01-02 15:16:27.730   
3  EAA2485BA64710D3   classic_bike  2025-01-23 08:49:05.814   
4  7F8BE2471C7F746B  electric_bike  2025-01-16 08:38:32.338   

                  ended_at            start_station_name start_station_id  \
0  2025-01-21 17:37:52.015     Wacker Dr & Washington St     KA1503000072   
1  2025-01-11 15:49:11.139   Halsted St & Wrightwood Ave     TA1309000061   
2  2025-01-02 15:28:03.230  Southport Ave & Waveland Ave            13235   
3  2025-01-23 08:52:40.047  Southport Ave & Waveland Ave            13235   
4  2025-01-16 08:41:06.767  Southport Ave & Waveland Ave            13235   

            end_station_name end_station_id  start_lat  start_lng    end_lat  \
0       McClurg Ct & Ohio St   TA1306000029  41.883143 -87.637242  41.892592  

In [10]:
print(combined_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2141425 entries, 0 to 2141424
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 212.4+ MB
None


In [11]:
# Null values will be handled in this part:
print("Shape of the combined DataFrame:", combined_df.shape)
print("\nMissing values (NaN/NaT) per column:")
print(combined_df.isnull().sum())

print("\nPercentage of missing values per column:")
print((combined_df.isnull().sum() / len(combined_df)) * 100)

Shape of the combined DataFrame: (2141425, 13)

Missing values (NaN/NaT) per column:
ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    429577
start_station_id      429577
end_station_name      447149
end_station_id        447149
start_lat                  0
start_lng                  0
end_lat                 2187
end_lng                 2187
member_casual              0
dtype: int64

Percentage of missing values per column:
ride_id                0.000000
rideable_type          0.000000
started_at             0.000000
ended_at               0.000000
start_station_name    20.060334
start_station_id      20.060334
end_station_name      20.880909
end_station_id        20.880909
start_lat              0.000000
start_lng              0.000000
end_lat                0.102128
end_lng                0.102128
member_casual          0.000000
dtype: float64


In [12]:
# Critical columns: ride_id, rideable_type, started_at, ended_at and member_casual are very important in this analysis. Thankfully, none is null.

In [14]:
# Normally, we could fix values at column end_lat and end_lng to 0 but distance based analysis,
# Could give us some hidden information about users preference on membership type. 
# Thus, minor amount of null values in those columns will be removed.

In [15]:
# Create copy of dataframe to handle cleaning without any data loss
df_cleaned = combined_df.copy()

In [16]:
columns_to_drop = ['end_lat', 'end_lng']
df_cleaned.dropna(subset=columns_to_drop, inplace=True)

In [19]:
# Check if null values dropped:
print("Percentage of missing values per column:")
print((df_cleaned.isnull().sum() / len(combined_df)) * 100)

Percentage of missing values per column:
ride_id                0.000000
rideable_type          0.000000
started_at             0.000000
ended_at               0.000000
start_station_name    20.060334
start_station_id      20.060334
end_station_name      20.778780
end_station_id        20.778780
start_lat              0.000000
start_lng              0.000000
end_lat                0.000000
end_lng                0.000000
member_casual          0.000000
dtype: float64


In [23]:
# Handle nulls in station information columns:
# These often have a lot of nulls, dropping them is not practical.
# We'll fill them with 'UNKNOWN' to keep the rows.
station_columns = ['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id']

for col in ['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id']:
    if col in df_cleaned.columns:
        df_cleaned[col] = df_cleaned[col].fillna('UNKNOWN')

In [24]:
# Verify nulls after handling
print("\nMissing values after initial null handling:")
print(df_cleaned.isnull().sum())
print("\nPercentage of missing values after initial null handling:")
print((df_cleaned.isnull().sum() / len(df_cleaned)) * 100)


Missing values after initial null handling:
ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

Percentage of missing values after initial null handling:
ride_id               0.0
rideable_type         0.0
started_at            0.0
ended_at              0.0
start_station_name    0.0
start_station_id      0.0
end_station_name      0.0
end_station_id        0.0
start_lat             0.0
start_lng             0.0
end_lat               0.0
end_lng               0.0
member_casual         0.0
dtype: float64


In [26]:
# Data types handling:
print("Dtypes before conversion:")
print(df_cleaned.dtypes)

Dtypes before conversion:
ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object


In [27]:
# started_at and ended_at columns should be converted to datetime format:
# Using errors='coerce' will turn any unparseable dates into NaT (Not a Time),
# which is good for identifying and potentially handling malformed dates.
df_cleaned['started_at'] = pd.to_datetime(df_cleaned['started_at'], errors='coerce')
df_cleaned['ended_at'] = pd.to_datetime(df_cleaned['ended_at'], errors='coerce')

In [28]:
# Lets check if converting is done without any errors. Problematic convertions should be appear as null values:
print("\nMissing values after initial dtype handling:")
print(df_cleaned.isnull().sum())


Missing values after initial dtype handling:
ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64


In [29]:
# Convert 'rideable_type' and 'member_casual' to 'category'
categorical_cols = ['rideable_type', 'member_casual']
for col in categorical_cols:
    if col in df_cleaned.columns:
        df_cleaned[col] = df_cleaned[col].astype('category')

In [30]:
# Lets check if converting is done without any errors. Problematic convertions should be appear as null values:
print("\nMissing values after initial dtype handling:")
print(df_cleaned.isnull().sum())


Missing values after initial dtype handling:
ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64


In [31]:
# Convert other 'object' columns to 'string' dtype
string_cols = [
    'ride_id',
    'start_station_name',
    'start_station_id',
    'end_station_name',
    'end_station_id'
]
for col in string_cols:
    if col in df_cleaned.columns and df_cleaned[col].dtype == 'object':
        df_cleaned[col] = df_cleaned[col].astype('string')

print("\n\nDtypes after conversion:")
print(df_cleaned.dtypes)

# Verify any NaT values
print("\nMissing values after type conversion (checking for NaT if coerce was used):")
print(df_cleaned.isnull().sum())



Dtypes after conversion:
ride_id               string[python]
rideable_type               category
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name    string[python]
start_station_id      string[python]
end_station_name      string[python]
end_station_id        string[python]
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual               category
dtype: object

Missing values after type conversion (checking for NaT if coerce was used):
ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64


In [32]:
# Duplicate handling is crucial before importing our dataset to sql:

print("Shape of DataFrame before checking for duplicates:", df_cleaned.shape)

# Check for duplicate ride_ids
# Keep=False marks ALL occurrences of a duplicate as True
duplicate_ride_ids = df_cleaned[df_cleaned.duplicated(subset=['ride_id'], keep=False)]

if not duplicate_ride_ids.empty:
    print(f"\n--- Found {len(duplicate_ride_ids)} rows with duplicate ride_ids. ---")
    print("\nSample of duplicate ride_ids (showing all occurrences):")
    print(duplicate_ride_ids.sort_values('ride_id').head(10)) # Display first 10 duplicate rows
else:
    print("\nNo duplicate ride_ids found. Dataset is unique based on ride_id.")

Shape of DataFrame before checking for duplicates: (2139238, 13)

No duplicate ride_ids found. Dataset is unique based on ride_id.


In [33]:
# There are no duplicates in our primary key column. This is great. Now we can move on to data manipulation to handle outliers.

In [34]:
print("Shape of DataFrame before handling ride duration outliers:", df_cleaned.shape)

# 1. Calculate ride_length in minutes
df_cleaned['ride_length_minutes'] = (df_cleaned['ended_at'] - df_cleaned['started_at']).dt.total_seconds() / 60

# Check for negative or zero ride lengths (should ideally be handled by >= 1 min filter, but good to inspect)
negative_rides = df_cleaned[df_cleaned['ride_length_minutes'] <= 0]
if not negative_rides.empty:
    print(f"\nFound {len(negative_rides)} rides with non-positive duration. These will be removed.")
    
# 2. Define outlier thresholds
min_ride_minutes = 1  # 1 minute
max_ride_minutes = 1440 # 24 hours

# 3. Filter the DataFrame to keep only valid rides
initial_rides = df_cleaned.shape[0]
df_cleaned = df_cleaned[
    (df_cleaned['ride_length_minutes'] >= min_ride_minutes) &
    (df_cleaned['ride_length_minutes'] <= max_ride_minutes)
]
rides_after_duration_filter = df_cleaned.shape[0]

print(f"\nRemoved {initial_rides - rides_after_duration_filter} rides due to duration outliers.")
print(f"New DataFrame shape after duration filtering: {df_cleaned.shape}")

# Quick check on min/max duration
print(f"\nMin ride duration after filtering: {df_cleaned['ride_length_minutes'].min():.2f} minutes")
print(f"Max ride duration after filtering: {df_cleaned['ride_length_minutes'].max():.2f} minutes")

Shape of DataFrame before handling ride duration outliers: (2139238, 13)

Removed 52106 rides due to duration outliers.
New DataFrame shape after duration filtering: (2087132, 14)

Min ride duration after filtering: 1.00 minutes
Max ride duration after filtering: 1435.21 minutes


In [35]:
# Now that outliers in ride lenght are handled, we can move on to handle very long, unrealistic rides to handle them. 

In [39]:
print("\nShape of DataFrame before handling geographical outliers:", df_cleaned.shape)

# Approximate geographical bounds for Chicago area
chicago_lat_min = 41.5
chicago_lat_max = 42.5
chicago_lng_min = -88.0
chicago_lng_max = -87.0

initial_rides = df_cleaned.shape[0]

# Filter out rides where start or end coordinates are outside reasonable Chicago bounds
df_cleaned = df_cleaned[
    (df_cleaned['start_lat'] >= chicago_lat_min) & (df_cleaned['start_lat'] <= chicago_lat_max) &
    (df_cleaned['start_lng'] >= chicago_lng_min) & (df_cleaned['start_lng'] <= chicago_lng_max) &
    (df_cleaned['end_lat'] >= chicago_lat_min) & (df_cleaned['end_lat'] <= chicago_lat_max) &
    (df_cleaned['end_lng'] >= chicago_lng_min) & (df_cleaned['end_lng'] <= chicago_lng_max)
]

rides_after_geo_filter = df_cleaned.shape[0]

print(f"\nRemoved {initial_rides - rides_after_geo_filter} rides due to geographical outliers.")
print(f"New DataFrame shape after geographical filtering: {df_cleaned.shape}")


Shape of DataFrame before handling geographical outliers: (2087132, 14)

Removed 2 rides due to geographical outliers.
New DataFrame shape after geographical filtering: (2087130, 14)


In [40]:
# Lets handle outliers in bike types and membership types:
# Ideally there should be two types of bikes and two types of memberships

print("\n--- Category Dtype Entry Check ---")

# Check 'rideable_type'
print("\nUnique values and counts for 'rideable_type':")
print(df_cleaned['rideable_type'].value_counts(dropna=False))

# Check 'member_casual'
print("\nUnique values and counts for 'member_casual':")
print(df_cleaned['member_casual'].value_counts(dropna=False))


--- Category Dtype Entry Check ---

Unique values and counts for 'rideable_type':
rideable_type
electric_bike    1297179
classic_bike      789951
Name: count, dtype: int64

Unique values and counts for 'member_casual':
member_casual
member    1394304
casual     692826
Name: count, dtype: int64


In [41]:
# Lets check the dataset
print(df_cleaned.head())

            ride_id  rideable_type              started_at  \
0  7569BC890583FCD7   classic_bike 2025-01-21 17:23:54.538   
1  013609308856B7FC  electric_bike 2025-01-11 15:44:06.795   
2  EACACD3CE0607C0D   classic_bike 2025-01-02 15:16:27.730   
3  EAA2485BA64710D3   classic_bike 2025-01-23 08:49:05.814   
4  7F8BE2471C7F746B  electric_bike 2025-01-16 08:38:32.338   

                 ended_at            start_station_name start_station_id  \
0 2025-01-21 17:37:52.015     Wacker Dr & Washington St     KA1503000072   
1 2025-01-11 15:49:11.139   Halsted St & Wrightwood Ave     TA1309000061   
2 2025-01-02 15:28:03.230  Southport Ave & Waveland Ave            13235   
3 2025-01-23 08:52:40.047  Southport Ave & Waveland Ave            13235   
4 2025-01-16 08:41:06.767  Southport Ave & Waveland Ave            13235   

            end_station_name end_station_id  start_lat  start_lng    end_lat  \
0       McClurg Ct & Ohio St   TA1306000029  41.883143 -87.637242  41.892592   
1   Racine

In [42]:
# As we can see, ride_lenght_minutes part is not so easy to read. We can round the values to only have 1 numerical after the delimiter.

In [45]:
# First, lets change the column name:
if 'ride_length_minutes' in df_cleaned.columns and 'trip_duration_mins' not in df_cleaned.columns:
    df_cleaned.rename(columns={'ride_length_minutes': 'trip_duration_mins'}, inplace=True)
    print("Renamed 'ride_length_minutes' to 'trip_duration_mins'.")
else:
    print("'trip_duration_mins' column already exists.")

print("\nFirst 5 rows with 'trip_duration_mins':")
print(df_cleaned[['started_at', 'ended_at', 'trip_duration_mins']].head())

Renamed 'ride_length_minutes' to 'trip_duration_mins'.

First 5 rows with 'trip_duration_mins':
               started_at                ended_at  trip_duration_mins
0 2025-01-21 17:23:54.538 2025-01-21 17:37:52.015           13.957950
1 2025-01-11 15:44:06.795 2025-01-11 15:49:11.139            5.072400
2 2025-01-02 15:16:27.730 2025-01-02 15:28:03.230           11.591667
3 2025-01-23 08:49:05.814 2025-01-23 08:52:40.047            3.570550
4 2025-01-16 08:38:32.338 2025-01-16 08:41:06.767            2.573817


In [46]:
# Round the values in the 'trip_duration_mins' column to 1 decimal place
df_cleaned['trip_duration_mins'] = df_cleaned['trip_duration_mins'].round(1)

print("Updated 'trip_duration_mins' column (first 5 rows):")
print(df_cleaned[['started_at', 'ended_at', 'trip_duration_mins']].head())

print(f"\nMin duration (rounded): {df_cleaned['trip_duration_mins'].min()} minutes")
print(f"Max duration (rounded): {df_cleaned['trip_duration_mins'].max()} minutes")

Updated 'trip_duration_mins' column (first 5 rows):
               started_at                ended_at  trip_duration_mins
0 2025-01-21 17:23:54.538 2025-01-21 17:37:52.015                14.0
1 2025-01-11 15:44:06.795 2025-01-11 15:49:11.139                 5.1
2 2025-01-02 15:16:27.730 2025-01-02 15:28:03.230                11.6
3 2025-01-23 08:49:05.814 2025-01-23 08:52:40.047                 3.6
4 2025-01-16 08:38:32.338 2025-01-16 08:41:06.767                 2.6

Min duration (rounded): 1.0 minutes
Max duration (rounded): 1435.2 minutes


In [48]:
output_folder = 'cleaned_data/'
output_file_csv = os.path.join(output_folder, 'cyclistic_trips_cleaned_temp.csv') # Use a temp name

print(f"Saving df_cleaned to {output_file_csv} before kernel restart...")
df_cleaned.to_csv(output_file_csv, index=False) # index=False is important
print("df_cleaned saved to CSV successfully.")

Saving df_cleaned to cleaned_data/cyclistic_trips_cleaned_temp.csv before kernel restart...
df_cleaned saved to CSV successfully.


In [49]:
import sys

# Install pyarrow
print("Installing pyarrow...")
!{sys.executable} -m pip install pyarrow

# Install geopy
print("Installing geopy...")
!{sys.executable} -m pip install geopy

print("Installation commands sent. You may need to restart the kernel now.")

Installing pyarrow...
   ---------------------------------------- 0.0/26.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/26.1 MB ? eta -:--:--
   -- ------------------------------------- 1.6/26.1 MB 7.0 MB/s eta 0:00:04
   -- ------------------------------------- 1.8/26.1 MB 7.3 MB/s eta 0:00:04
   ------- -------------------------------- 4.7/26.1 MB 7.6 MB/s eta 0:00:03
   ---------- ----------------------------- 6.6/26.1 MB 7.9 MB/s eta 0:00:03
   ------------ --------------------------- 8.4/26.1 MB 8.1 MB/s eta 0:00:03
   -------------- ------------------------- 9.7/26.1 MB 8.2 MB/s eta 0:00:02
   ------------------ --------------------- 12.1/26.1 MB 8.4 MB/s eta 0:00:02
   --------------------- ------------------ 13.9/26.1 MB 8.4 MB/s eta 0:00:02
   ------------------------ --------------- 15.7/26.1 MB 8.5 MB/s eta 0:00:02
   -------------------------- ------------- 17.6/26.1 MB 8.5 MB/s eta 0:00:01
   ----------------------------- ---------- 19.4/26.1 MB 8.6 MB/


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Installation commands sent. You may need to restart the kernel now.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import os

output_folder = 'cleaned_data/'
output_file_csv = os.path.join(output_folder, 'cyclistic_trips_cleaned_temp.csv')

print(f"Loading df_cleaned from {output_file_csv} after kernel restart...")
df_cleaned = pd.read_csv(output_file_csv, low_memory=False) # low_memory=False is good practice

# Re-apply critical type conversions if they were not saved correctly by to_csv
# (datetime columns are often read back as strings from CSV)
df_cleaned['started_at'] = pd.to_datetime(df_cleaned['started_at'], errors='coerce')
df_cleaned['ended_at'] = pd.to_datetime(df_cleaned['ended_at'], errors='coerce')

# Re-apply category conversions if they were not saved correctly by to_csv
categorical_cols = ['rideable_type', 'member_casual']
for col in categorical_cols:
    if col in df_cleaned.columns:
        df_cleaned[col] = df_cleaned[col].astype('category')

# Re-apply string conversions if they were not saved correctly by to_csv
string_cols = [
    'ride_id',
    'start_station_name',
    'start_station_id',
    'end_station_name',
    'end_station_id'
]
for col in string_cols:
    if col in df_cleaned.columns and df_cleaned[col].dtype == 'object':
        df_cleaned[col] = df_cleaned[col].astype('string')

print("df_cleaned loaded successfully from CSV.")
print(df_cleaned.info()) # Verify types and data

Loading df_cleaned from cleaned_data/cyclistic_trips_cleaned_temp.csv after kernel restart...
df_cleaned loaded successfully from CSV.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2087130 entries, 0 to 2087129
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             string        
 1   rideable_type       category      
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  string        
 5   start_station_id    string        
 6   end_station_name    string        
 7   end_station_id      string        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       category      
 13  trip_duration_mins  float64       
dtypes: category(2), datetime64[ns](2), float64(5), string(5)
memory usage: 195.1 MB
None


In [2]:
from geopy.distance import geodesic
import pandas as pd

print("\n--- Starting distance calculation for 'distance_traveled_km' ---")
print("Shape of DataFrame before distance calculation:", df_cleaned.shape)

# Create an empty column for distance, initialized with Pandas' NA (nullable type)
# This allows us to explicitly mark rows where distance couldn't be calculated.
df_cleaned['distance_traveled_km'] = pd.NA

# It's good practice to ensure lat/lng are numeric (float) before calculations

df_cleaned['start_lat'] = pd.to_numeric(df_cleaned['start_lat'], errors='coerce')
df_cleaned['start_lng'] = pd.to_numeric(df_cleaned['start_lng'], errors='coerce')
df_cleaned['end_lat'] = pd.to_numeric(df_cleaned['end_lat'], errors='coerce')
df_cleaned['end_lng'] = pd.to_numeric(df_cleaned['end_lng'], errors='coerce')

# Count rows with potentially invalid (0 or NaN) coordinates before calculation
# These rows won't yield a valid distance.
invalid_coords_mask = (
    (df_cleaned['start_lat'].isna()) | (df_cleaned['start_lng'].isna()) |
    (df_cleaned['end_lat'].isna()) | (df_cleaned['end_lng'].isna()) |
    (df_cleaned['start_lat'] == 0.0) | (df_cleaned['start_lng'] == 0.0) |
    (df_cleaned['end_lat'] == 0.0) | (df_cleaned['end_lng'] == 0.0)
)
num_rows_with_invalid_coords = invalid_coords_mask.sum()

if num_rows_with_invalid_coords > 0:
    print(f"Warning: {num_rows_with_invalid_coords} rows have invalid (NaN or 0) coordinates. Distance will be marked as NA for these.")

# Define a function to calculate distance safely
def calculate_distance(row):
    # Check for valid coordinates (not NaN and not 0.0)
    if (pd.notna(row['start_lat']) and row['start_lat'] != 0.0 and
        pd.notna(row['start_lng']) and row['start_lng'] != 0.0 and
        pd.notna(row['end_lat']) and row['end_lat'] != 0.0 and
        pd.notna(row['end_lng']) and row['end_lng'] != 0.0):
        
        start_coords = (row['start_lat'], row['start_lng'])
        end_coords = (row['end_lat'], row['end_lng'])
        try:
            # Calculate distance in kilometers and round to 2 decimal places for readability
            distance_km = geodesic(start_coords, end_coords).km
            return round(distance_km, 2)
        except ValueError: # Catch potential errors from geodesic for very strange coords
            return pd.NA
    else:
        return pd.NA # Return NA if coordinates are invalid

# Apply the function to each row
# This operation can take some time for millions of rows.
print("Applying distance calculation (this might take a few minutes for a large dataset)...")
df_cleaned['distance_traveled_km'] = df_cleaned.apply(calculate_distance, axis=1)

# Report on the results
num_calculated_distances = df_cleaned['distance_traveled_km'].count() # count non-NA values
print(f"\nSuccessfully calculated distance for {num_calculated_distances} rides.")
print(f"Number of rides where distance could not be calculated (marked as NA): {df_cleaned['distance_traveled_km'].isna().sum()}")

print("\nFirst 10 rows with new 'distance_traveled_km' column:")
print(df_cleaned[['start_lat', 'start_lng', 'end_lat', 'end_lng', 'distance_traveled_km']].head(10))

print(f"\nSummary of 'distance_traveled_km':")
print(df_cleaned['distance_traveled_km'].describe())

# A quick check on minimum distance. 0km is possible if start and end coords are identical
if df_cleaned['distance_traveled_km'].min() == 0.0:
    print("\nNote: Some rides have a 0.0 km distance, indicating start and end points are the same.")


--- Starting distance calculation for 'distance_traveled_km' ---
Shape of DataFrame before distance calculation: (2087130, 14)
Applying distance calculation (this might take a few minutes for a large dataset)...

Successfully calculated distance for 2087130 rides.
Number of rides where distance could not be calculated (marked as NA): 0

First 10 rows with new 'distance_traveled_km' column:
   start_lat  start_lng    end_lat    end_lng  distance_traveled_km
0  41.883143 -87.637242  41.892592 -87.617289                  1.96
1  41.929147 -87.649153  41.939743 -87.658865                  1.43
2  41.948226 -87.664071  41.945529 -87.646439                  1.49
3  41.948226 -87.664071  41.943739 -87.664020                  0.50
4  41.948226 -87.664071  41.943739 -87.664020                  0.50
5  41.853780 -87.646603  41.857506 -87.645991                  0.42
6  41.929143 -87.649077  41.928773 -87.663913                  1.23
7  41.917741 -87.691392  41.882409 -87.639767                 

In [3]:
print(df_cleaned.head())

            ride_id  rideable_type              started_at  \
0  7569BC890583FCD7   classic_bike 2025-01-21 17:23:54.538   
1  013609308856B7FC  electric_bike 2025-01-11 15:44:06.795   
2  EACACD3CE0607C0D   classic_bike 2025-01-02 15:16:27.730   
3  EAA2485BA64710D3   classic_bike 2025-01-23 08:49:05.814   
4  7F8BE2471C7F746B  electric_bike 2025-01-16 08:38:32.338   

                 ended_at            start_station_name start_station_id  \
0 2025-01-21 17:37:52.015     Wacker Dr & Washington St     KA1503000072   
1 2025-01-11 15:49:11.139   Halsted St & Wrightwood Ave     TA1309000061   
2 2025-01-02 15:28:03.230  Southport Ave & Waveland Ave            13235   
3 2025-01-23 08:52:40.047  Southport Ave & Waveland Ave            13235   
4 2025-01-16 08:41:06.767  Southport Ave & Waveland Ave            13235   

            end_station_name end_station_id  start_lat  start_lng    end_lat  \
0       McClurg Ct & Ohio St   TA1306000029  41.883143 -87.637242  41.892592   
1   Racine

In [4]:
output_folder = 'cleaned_data/'
final_parquet_path = os.path.join(output_folder, 'cyclistic_trips_cleaned.parquet')

print(f"Saving df_cleaned to Parquet file: {final_parquet_path}...")

# Save the DataFrame to Parquet
# index=False is important to prevent Pandas from writing its DataFrame index as a column in the Parquet file
df_cleaned.to_parquet(final_parquet_path, index=False)

print("df_cleaned successfully saved to Parquet.")
print(f"You can find the file at: {os.path.abspath(final_parquet_path)}")

Saving df_cleaned to Parquet file: cleaned_data/cyclistic_trips_cleaned.parquet...
df_cleaned successfully saved to Parquet.
You can find the file at: C:\Cyclistic_Capstone\cleaned_data\cyclistic_trips_cleaned.parquet


In [6]:
import sys
print("Installing pyodbc...")
!{sys.executable} -m pip install pyodbc

print("Installing sqlalchemy...")
!{sys.executable} -m pip install sqlalchemy

print("Installation commands sent. You may need to restart the kernel now.")

Installing pyodbc...
Installing sqlalchemy...



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----------------------------- ---------- 1.6/2.1 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 5.0 MB/s eta 0:00:00
Installation commands sent. You may need to restart the kernel now.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import os

# Define the output folder and file path where Parquet file is located
output_folder = 'cleaned_data/'
final_parquet_path = os.path.join(output_folder, 'cyclistic_trips_cleaned.parquet')

# Check if the file exists before trying to load it
if not os.path.exists(final_parquet_path):
    print(f"Error: Parquet file not found at {final_parquet_path}")
    print("Please ensure the file was saved correctly and the path is accurate.")
else:
    print(f"Loading df_cleaned from Parquet file: {final_parquet_path}...")

    # Read the Parquet file into a DataFrame
    df_cleaned = pd.read_parquet(final_parquet_path)

    print("df_cleaned successfully loaded from Parquet.")

    # Display basic info to verify data types and integrity
    print("\nDataFrame Info after loading from Parquet:")
    print(df_cleaned.info())

    print("\nFirst 5 rows of loaded DataFrame:")
    print(df_cleaned.head())

Loading df_cleaned from Parquet file: cleaned_data/cyclistic_trips_cleaned.parquet...
df_cleaned successfully loaded from Parquet.

DataFrame Info after loading from Parquet:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2087130 entries, 0 to 2087129
Data columns (total 15 columns):
 #   Column                Dtype         
---  ------                -----         
 0   ride_id               string        
 1   rideable_type         category      
 2   started_at            datetime64[ns]
 3   ended_at              datetime64[ns]
 4   start_station_name    string        
 5   start_station_id      string        
 6   end_station_name      string        
 7   end_station_id        string        
 8   start_lat             float64       
 9   start_lng             float64       
 10  end_lat               float64       
 11  end_lng               float64       
 12  member_casual         category      
 13  trip_duration_mins    float64       
 14  distance_traveled_km  float64      

In [2]:
# Now we can move on to importing our dataset to Microsoft SQL Server

In [4]:
import pyodbc
from sqlalchemy import create_engine
import urllib
import pandas as pd

In [7]:
# --- Configuration for SQL Server Connection (Windows Authentication) ---
server = r'EREN_MONSTER\EREN' # Fixed SyntaxWarning here
database = 'CYCLISTICBIKESHARE'
table_name = 'BikeTrips'

# --- Create SQL Alchemy Engine ---
conn_str = (
    f'DRIVER={{ODBC Driver 18 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'Trusted_Connection=yes;'
    f'TrustServerCertificate=yes;'
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")

# --- Data Import ---
print(f"Attempting to load {df_cleaned.shape[0]} rows into SQL Server table '{table_name}' in database '{database}' using Windows Authentication...")

try:
    df_cleaned.to_sql(table_name, con=engine, if_exists='append', index=False, chunksize=1000)

    print(f"Successfully loaded {df_cleaned.shape[0]} rows into SQL Server.")

except Exception as e:
    print(f"Error loading data to SQL Server: {e}")
    print("\nTroubleshooting tips for Windows Authentication:")
    print("1. Ensure SQL Server is running and accessible.")
    print("2. Verify your 'server' name is correct (e.g., 'localhost', '.\SQLEXPRESS').")
    print("3. Check that 'ODBC Driver 17 for SQL Server' is installed on your system.")
    print("4. Ensure the table schema in SQL Server exactly matches your DataFrame columns and types.")
    print("5. Verify the Windows user account running this Jupyter Notebook has permissions to connect to SQL Server and write to the database/table.")
    print("6. Check firewall settings if SQL Server is on a different machine.")
finally:
    engine.dispose()

<>:29: SyntaxWarning: invalid escape sequence '\S'
<>:29: SyntaxWarning: invalid escape sequence '\S'
C:\Users\eren_\AppData\Local\Temp\ipykernel_18980\4108179022.py:29: SyntaxWarning: invalid escape sequence '\S'
  print("2. Verify your 'server' name is correct (e.g., 'localhost', '.\SQLEXPRESS').")


Attempting to load 2087130 rows into SQL Server table 'BikeTrips' in database 'CYCLISTICBIKESHARE' using Windows Authentication...
Successfully loaded 2087130 rows into SQL Server.
